# Settings

In [1]:
import sys
import os

os.chdir('..')
os.getcwd()

'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

In [2]:
import os
os.getcwd()


'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

# Imports

In [3]:
from scripts.model import CharBiLSTM, prepare_data
from scripts.data import create_data_loader, load_data
from scripts.preprocessing import get_typoglycemia_modified_data, sentence_tokennizer, tokenize_dataframe, get_max_length
from scripts.baseline import get_base_line_score

from torch import nn, optim
import torch

from sklearn.model_selection import train_test_split

import editdistance
from tqdm.autonotebook import trange, tqdm
import random 

from sklearn.metrics import f1_score, accuracy_score
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

c:\Users\ruben\OneDrive\Dokumenter\GitHub\ANLP-Project\scripts\preprocessing.py:56: SyntaxWarning: invalid escape sequence '\s'
  x = re.sub('[^a-zA-Z\s\.]', '', x)
c:\Users\ruben\OneDrive\Dokumenter\GitHub\ANLP-Project\scripts\preprocessing.py:58: SyntaxWarning: invalid escape sequence '\.'
  x = re.sub('\.{2,}', '.', x)
c:\Users\ruben\OneDrive\Dokumenter\GitHub\ANLP-Project\scripts\preprocessing.py:62: SyntaxWarning: invalid escape sequence '\s'
  x = x = re.sub('\s{2,}', ' ', x)
C:\Users\ruben\AppData\Local\Temp\ipykernel_16216\2098325508.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


# Easy

In [4]:
from model_train_easy import device, complexity_level, max_length, batch_size, input_size, hidden_size, output_size, num_layers, complexity_level, model, loss_function, optimizer, epochs

cuda:0
True


In [5]:
device, complexity_level

(device(type='cuda', index=0), 'Easy')

In [6]:
train_loader, validation_loader, test_loader = prepare_data(complexity_level = complexity_level,
                                                            max_length = max_length,
                                                            batch_size = batch_size)

In [7]:
for sample in train_loader:
    X, y = sample
    print(X.shape, y.shape)
    print(np.unique(y.cpu(), return_counts=True))
    print(X.device)
    break

torch.Size([512, 151]) torch.Size([512, 151])
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64), array([42657,  2653,   492,   899,  1064,  3384,   623,   522,  1453,
        2152,    95,   190,  1182,   753,  2178,  2012,   555,    16,
        1848,  2030,  2313,   759,   251,   538,    43,   422,    33,
        6195], dtype=int64))
cpu


In [8]:
os.getcwd()

'c:\\Users\\ruben\\OneDrive\\Dokumenter\\GitHub\\ANLP-Project'

In [9]:
if complexity_level.lower() == 'easy':
    PATH = f'models/{complexity_level.lower()}/model_1154.pt'
else:
    PATH = f'models/{complexity_level.lower()}/model_987.pt'

model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

CharBiLSTM(
  (lstm): LSTM(1, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=29, bias=True)
)

In [10]:
def index_to_char(index, target = True):
    if target == False:
        index = round(index*100, 0)
    index = int(index)
    if 1 <= index <= 25:
        return chr(index + ord('a') - 1), index
    elif index == 26:
        return ' ', index
    elif index == 27:
        return "<SENTSTOP>", index
    else:
        return "_", index  # for all other values, return None
    

# def index_to_char(index, target = True):
#     if target == False:
#         index = round(index*100, 0)
#     index = int(index)
#     if 1 <= index <= 26:
#         return chr(index + ord('a') - 1), index
#     elif index == 27:
#         return ' ', index
#     elif index == 28:
#         return "<SENTSTOP>", index
#     else:
#         return "_", index  # for all other values, return None

In [69]:
import csv
import editdistance

def get_metrics(loader, model, loader_str, output_file="output.csv", sample_size=10):
    preds = []
    labels = []
    inputs = []  # To store input sequences
    
    model.eval()
    loss = 0.0
    csv_data = [["Input Word", "Label_Input_Editdistance", "Label Word", "Pred_label_Editdistance", "Predicted Word"]]  # CSV header
    sentence_label = []
    sentence_pred = []
    sentence_input = []  # To store input words
    all_edit_distances = []
    print_count = 0
    all_typo_edit_distances = []
    all_edit_sentence_distances = []
    all_typo_edit_sentence_distances = []

    with torch.no_grad():
        for batch in tqdm(loader, position=3, leave=False):
            typo_batch, sentence_batch = batch  # typo_batch is the input, sentence_batch is the target sequence
            typo_batch = typo_batch.to(device)
            sentence_batch =sentence_batch.to(device)
            
            sentence_batch = sentence_batch.reshape(-1, max_length, 1)
            typo_batch = typo_batch.reshape(-1, max_length, 1)
            
            # print(sentence_batch[0])
            # for index in range(len(sentence_batch)):
            #     sentence = sentence_batch[index].cpu().numpy().reshape(-1)
            #     typo = typo_batch[index].cpu().numpy().reshape(-1)

            #     for i, value in enumerate(sentence):
            #         if value == -1:
            #             sentence[i] = 26
            #             break

            #     for i, value in enumerate(typo):
            #         if value == -1:
            #             typo[i] = 26
            #             break
                
            #     sentence_batch[index] = torch.tensor(sentence).reshape(max_length, 1).to(device)
            #     typo_batch[index] = torch.tensor(typo).reshape(max_length, 1).to(device)
            # print(sentence_batch[0])
                
            # print(typo_batch.shape)
            y = model.forward(typo_batch, train=False)

            
            sentence_batch_loss = sentence_batch.reshape(-1)
            # Calculate and accumulate loss
            loss_batch = loss_function(y, sentence_batch_loss)
            loss += loss_batch.item()
            
            

            word_label = []
            word_pred = []
            word_input = []
            
            y = y.reshape(-1, max_length, output_size)
            ### Make it work for the nabsew dimensionality
            for sentence_index in range(len(y)):
                sentence = y[sentence_index]
                sentence_labels = sentence_batch[sentence_index]
                sentence_inputs = typo_batch[sentence_index]
                
                sentence_edit_distances = []
                typo_sentence_edit_distances = []


                # Get predictions, labels, and inputs in a flattened form
                batch_preds = torch.argmax(sentence, dim=1).cpu().numpy().reshape(-1)
                batch_labels = sentence_labels.cpu().numpy().reshape(-1)
                batch_inputs = sentence_inputs.cpu().numpy().reshape(-1)
                
                # Append these results for later accuracy and F1 calculations
                preds.extend(batch_preds)
                labels.extend(batch_labels)
                inputs.extend(batch_inputs)
                # print("label_char", "pred_char", "input_char")
                
                # Process each element in the batch
                for label_idx in range(len(batch_preds)):
                    label_char, label_index_ = index_to_char(batch_labels[label_idx])
                    pred_char, pred_index_ = index_to_char(batch_preds[label_idx])
                    input_char, input_index_ = index_to_char(batch_inputs[label_idx],target = False)

                    # print(f"label_index_: {label_index_}, pred_index_: {pred_index_}, input_index_: {input_index_}")

                    # Ignore padding characters
                    # if label_char != "_":
                    #     word_label.append(label_char)
                    # if pred_char != "_":
                    #     word_pred.append(pred_char)
                    # if input_char != "_":
                    #     word_input.append(input_char)
                    
                    if label_char not in ["_", " "]:
                        word_label.append(label_char)
                    if pred_char not in ["_", " "]:
                        word_pred.append(pred_char)
                    if input_char not in ["_", " "]:
                        word_input.append(input_char)
 
 

                    # If end of a word (detected by space), append word to csv_data
                    # print("Label", label_char, label_index_, "Input", input_char, input_index_, "Pred", pred_char, pred_index_)

                    if ((label_char == " " ) or (label_char == "<SENTSTOP>" )) and len(word_label) > 0:
                        # Join and append the completed word to csv data
                        
                        # remove any instances of "<SENTSTOP>" from the word_label
                        word_label = [char for char in word_label if char != "<SENTSTOP>"]
                        word_pred = [char for char in word_pred if char != "<SENTSTOP>"]
                        word_input = [char for char in word_input if char != "<SENTSTOP>"]
                        try:
                            distance = editdistance.eval("".join(word_pred), "".join(word_label)) / len("".join(word_label))
                            sentence_edit_distances.append(distance)
                        except:
                            pass
                        
                        try:
                            distance_typo_label = editdistance.eval("".join(word_input), "".join(word_label)) / len("".join(word_label))
                            typo_sentence_edit_distances.append(distance_typo_label)
                        except:
                            pass

                        if print_count == 0 and len(word_label) > 0:
                            print(f"Word input: {''.join(word_input)} Word pred: {''.join(word_pred)} Editdistance: {distance_typo_label}")
                            print(f"Word label: {''.join(word_label)} Word pred: {''.join(word_pred)} Editdistance: {distance}\n")
                        csv_data.append(["".join(word_input), distance_typo_label, "".join(word_label), distance, "".join(word_pred)])
                        # Clear the lists for the next word
                        word_label = []
                        word_pred = []
                        word_input = []
                
                try:
                    distance = editdistance.eval("".join(word_pred), "".join(word_label)) / len("".join(word_label))
                    sentence_edit_distances.append(distance)

                    distance_typo_label = editdistance.eval("".join(word_input), "".join(word_label)) / len("".join(word_label))
                    typo_sentence_edit_distances.append(distance_typo_label)
                    csv_data.append(["".join(word_input), distance_typo_label, "".join(word_label), distance, "".join(word_pred)])
                    
                except:
                    pass
                    # print(f"FAILED ON word_input '{word_input}'", "word_pred", word_pred)

                all_edit_distances.extend(sentence_edit_distances)
                all_edit_sentence_distances.append(np.mean(sentence_edit_distances))
                all_typo_edit_distances.extend(typo_sentence_edit_distances)
                all_typo_edit_sentence_distances.append(np.mean(typo_sentence_edit_distances))

                if print_count == 0 and len(word_label) > 0:
                    print(f"Word input: {''.join(word_input)} Word pred: {''.join(word_pred)} Editdistance: {distance_typo_label}")
                    print(f"Word label: {''.join(word_label)} Word pred: {''.join(word_pred)} Editdistance: {distance}\n")
                    print(f"Word label: {"".join(word_label)} Word pred: {"".join(word_pred)} Editdistance: {distance}\n")                   
                    print(f"All sentence edit distances", sentence_edit_distances)
                    print(f"Average Normalized Edit Distance Pred-Label: {sum(sentence_edit_distances) / len(sentence_edit_distances):.4f}")
                    print(f"Average Normalized Edit Distance Input-Label: {sum(typo_sentence_edit_distances) / len(typo_sentence_edit_distances):.4f}")
                word_label = []
                word_pred = []
                word_input = []
                # print_count += 1
            
                if sentence_index == 5:
                    break
            break
            

    # Write data to CSV file
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)
    
    # Display a subset of rows in the old format for console output
    print(f"\nDisplaying the first {sample_size} rows from the output file:\n")
    print("Input Word      | Label Word      | Predicted Word")
    print("-" * 50)
    for row in csv_data[1:sample_size + 1]:  # Skip header row, take only `sample_size` rows
        print(f"{row[0]:<15} | {row[2]:<15} | {row[4]}")
    
    # Compute and display metrics
    loss_avg = loss / len(loader)
    print(f"\n{loader_str} Loss: {loss_avg:.4f}")
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')


    # # ADD EDIT DISTANCE METRIC HERE
    
    
    print(f"\n{loader_str} Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    # Calculate average edit distance
    avg_edit_distance = sum(all_edit_distances) / len(all_edit_distances)
    avg_edit_distance_typo = sum(all_typo_edit_distances) / len(all_typo_edit_distances)
    print(f"\n{loader_str} Average Normalized Edit Distance Word-Level Pred-Label: {avg_edit_distance:.4f}")
    print(f"\n{loader_str} Average Normalized Edit Distance Word-Level Typo-Label: {avg_edit_distance_typo:.4f}")

    print(f"\n{loader_str} Average Normalized Edit Distance Sentence-Level Pred-Label: {np.mean(all_edit_sentence_distances)}")
    print(f"\n{loader_str} Average Normalized Edit Distance Sentence-Level Typo-Label: {np.mean(all_typo_edit_sentence_distances)}")


In [70]:
n_print = 100

In [71]:
# get_metrics(train_loader, model, "train", f"train_predictions_{complexity_level}.csv", n_print)

In [72]:
get_metrics(validation_loader, model, "validation", f"validation_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/77 [00:00<?, ?it/s]

Word input: mrurow Word pred: murruw Editdistance: 0.3333333333333333
Word label: murrow Word pred: murruw Editdistance: 0.16666666666666666

Word input: and Word pred: and Editdistance: 0.0
Word label: and Word pred: and Editdistance: 0.0

Word input: u Word pred: u Editdistance: 0.0
Word label: u Word pred: u Editdistance: 0.0

Word input: elishbteasd Word pred: esbsshsshed Editdistance: 0.7272727272727273
Word label: established Word pred: esbsshsshed Editdistance: 0.45454545454545453

Word input: rcok Word pred: rock Editdistance: 0.5
Word label: rock Word pred: rock Editdistance: 0.0

Word input: ciblnimg Word pred: bhambing Editdistance: 0.625
Word label: climbing Word pred: bhambing Editdistance: 0.375

Word input: roetus Word pred: seores Editdistance: 0.3333333333333333
Word label: routes Word pred: seores Editdistance: 0.6666666666666666

Word input: are Word pred: are Editdistance: 0.0
Word label: are Word pred: are Editdistance: 0.0

Word input: more Word pred: more Editdis

In [15]:
get_metrics(test_loader, model, "test", f"test_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/118 [00:00<?, ?it/s]

Word input: big Word pred: big Editdistance: 0.0
Word label: big Word pred: big Editdistance: 0.0

Word input: boi Word pred: bii Editdistance: 0.0
Word label: boi Word pred: bii Editdistance: 0.3333333333333333

Word input: tlod Word pred: sold Editdistance: 0.5
Word label: told Word pred: sold Editdistance: 0.25

Word input: his Word pred: his Editdistance: 0.0
Word label: his Word pred: his Editdistance: 0.0

Word input: fneird Word pred: fonnnd Editdistance: 0.6666666666666666
Word label: friend Word pred: fonnnd Editdistance: 0.5

Word input: diddy Word pred: didd Editdistance: 0.0
Word label: diddy Word pred: didd Editdistance: 0.2

Word input: abuot Word pred: about Editdistance: 0.4
Word label: about Word pred: about Editdistance: 0.0

Word input: mone Word pred: mone Editdistance: 0.0
Word label: mone Word pred: mone Editdistance: 0.0


Displaying the first 20 rows from the output file:

Input Word      | Label Word      | Predicted Word
---------------------------------------

# Model Analysis

In [46]:
df = pd.read_csv(f"{"validation"}_predictions_{complexity_level}.csv")
# Calculate word frequency, length, and average edit distance
df['Word'] = df['Input Word'].str.strip()
df['Frequency'] = df.groupby('Word')['Word'].transform('count')
df['Length'] = df['Word'].str.len()
df['Avg_Edit_Distance'] = df.groupby('Word')['Pred_label_Editdistance'].transform('mean')


df.groupby("Label Word")["Pred_label_Editdistance"].mean().mean()

0.48230250877332187

In [60]:
df = pd.read_csv(f"{"validation"}_predictions_{complexity_level}.csv")

df['Label Word Length'] = df['Label Word'].str.len()
grouped = df.groupby("Label Word").agg({"Label Word": "count", "Pred_label_Editdistance": "sum", "Label Word Length": "mean"}).rename({"Label Word": "Count", "Pred_label_Editdistance": "Total_Edit_Distance"}, axis=1)
grouped["mean"] = grouped["Total_Edit_Distance"] / grouped["Count"]
grouped["mean"].mean()


0.48230250877332187

In [ ]:
def analyze_predictions(data_type):
    if data_type not in ["validation", "test"]:
        raise ValueError("data_type must be either 'validation' or 'test'")
    
    df = pd.read_csv(f"{data_type}_predictions_{complexity_level}.csv")
    # Calculate word frequency, length, and average edit distance
    df['Word'] = df['Input Word'].str.strip()
    df['Frequency'] = df.groupby('Word')['Word'].transform('count')
    df['Length'] = df['Word'].str.len()
    df['Avg_Edit_Distance'] = df.groupby('Word')['Pred_label_Editdistance'].transform('mean')

    # Drop duplicates to keep unique words
    df_unique = df[['Word', 'Frequency', 'Length', 'Avg_Edit_Distance']].drop_duplicates()

    # Calculate correlations
    freq_avg_edit_corr = df_unique['Frequency'].corr(df_unique['Avg_Edit_Distance'])
    length_avg_edit_corr = df_unique['Length'].corr(df_unique['Avg_Edit_Distance'])

    print(f"Correlation between Frequency and Avg_Edit_Distance: {freq_avg_edit_corr}")
    print(f"Correlation between Length and Avg_Edit_Distance: {length_avg_edit_corr}")

    df_unique.to_csv(f"analysis/{data_type}_aggregated_{complexity_level}.csv", index=False)


analyze_predictions("validation")
analyze_predictions("test")


Correlation between Frequency and Avg_Edit_Distance: -0.017754464790179038
Correlation between Length and Avg_Edit_Distance: 0.053603651017553856
Correlation between Frequency and Avg_Edit_Distance: -0.01673991576951793
Correlation between Length and Avg_Edit_Distance: 0.04985416790346151


# Hard

In [17]:
from model_train_hard import device, complexity_level, max_length, batch_size, input_size, hidden_size, output_size, num_layers, complexity_level, model, loss_function, optimizer, epochs

cuda:0
True


In [18]:
device, complexity_level

(device(type='cuda', index=0), 'Hard')

In [19]:
train_loader, validation_loader, test_loader = prepare_data(complexity_level = complexity_level,
                                                            max_length = max_length,
                                                            batch_size = batch_size)

In [20]:
for sample in train_loader:
    X, y = sample
    print(X.shape, y.shape)
    print(np.unique(y.cpu(), return_counts=True))
    print(X.device)
    break

torch.Size([512, 151]) torch.Size([512, 151])
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64), array([32341,  3390,   592,  1218,  1492,  4320,   847,   645,  1798,
        2873,    94,   250,  1565,   956,  2857,  2710,   787,    28,
        2487,  2484,  3186,   959,   346,   632,    65,   571,    55,
        7764], dtype=int64))
cpu


In [21]:
if complexity_level.lower() == 'easy':
    PATH = f'models/{complexity_level.lower()}/model_1154.pt'
else:
    PATH = f'models/{complexity_level.lower()}/model_987.pt'

model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

CharBiLSTM(
  (lstm): LSTM(1, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=29, bias=True)
)

In [22]:
# get_metrics(train_loader, model, "train", f"train_predictions_{complexity_level}.csv", n_print)

In [23]:
get_metrics(validation_loader, model, "validation", f"validation_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/77 [00:00<?, ?it/s]

Word input: it Word pred: it Editdistance: 0.0
Word label: it Word pred: it Editdistance: 0.0

Word input: is Word pred: is Editdistance: 0.0
Word label: is Word pred: is Editdistance: 0.0

Word input: clltceauad Word pred: callecatad Editdistance: 0.6
Word label: calculated Word pred: callecatad Editdistance: 0.4

Word input: in Word pred: in Editdistance: 0.0
Word label: in Word pred: in Editdistance: 0.0

Word input: real Word pred: raal Editdistance: 0.0
Word label: real Word pred: raal Editdistance: 0.25

Word input: tmie Word pred: time Editdistance: 0.5
Word label: time Word pred: time Editdistance: 0.0

Word input: and Word pred: and Editdistance: 0.0
Word label: and Word pred: and Editdistance: 0.0

Word input: piseubhld Word pred: phblihhed Editdistance: 0.6666666666666666
Word label: published Word pred: phblihhed Editdistance: 0.2222222222222222

Word input: eervy Word pred: eesty Editdistance: 0.4
Word label: every Word pred: eesty Editdistance: 0.6

Word input: sdcones Wo

In [24]:
get_metrics(test_loader, model, "test", f"test_predictions_{complexity_level}.csv", n_print)

  0%|          | 0/96 [00:00<?, ?it/s]

Word input: koelrimets Word pred: koemeetees Editdistance: 0.6
Word label: kilometres Word pred: koemeetees Editdistance: 0.5

Word input: lnog Word pred: kong Editdistance: 0.5
Word label: long Word pred: kong Editdistance: 0.25

Word input: and Word pred: and Editdistance: 0.0
Word label: and Word pred: and Editdistance: 0.0

Word input: is Word pred: is Editdistance: 0.0
Word label: is Word pred: is Editdistance: 0.0

Word input: noted Word pred: noted Editdistance: 0.0
Word label: noted Word pred: noted Editdistance: 0.0

Word input: for Word pred: for Editdistance: 0.0
Word label: for Word pred: for Editdistance: 0.0

Word input: its Word pred: its Editdistance: 0.0
Word label: its Word pred: its Editdistance: 0.0

Word input: snweipeg Word pred: seeeeing Editdistance: 0.5
Word label: sweeping Word pred: seeeeing Editdistance: 0.25

Word input: coerrns Word pred: correes Editdistance: 0.5714285714285714
Word label: corners Word pred: correes Editdistance: 0.2857142857142857

Word 

# Model Analysis

In [25]:
analyze_predictions("validation")
analyze_predictions("test")

Correlation between Frequency and Avg_Edit_Distance: -0.017291266680612438
Correlation between Length and Avg_Edit_Distance: 0.06825150948150824
Correlation between Frequency and Avg_Edit_Distance: -0.016191566128198297
Correlation between Length and Avg_Edit_Distance: 0.05909745763675323
